In [24]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping


In [25]:
# 0) 설정(컬럼명 지정)
DATA_PATH = r"data\전처리\훈련데이터셋_날짜포함.csv"  
COL_GROUP  = "전체코드"
COL_DATE   = "날짜"
COL_TARGET = "파워"

WEATHER_COLS = [
    "최저기온(°C)",
    "3.0m 지중온도(°C)",
    "평균 현지기압(hPa)",
    "가조시간(hr)",
    "평균 상대습도(%)",
    "풍정합(100m)",
    "합계 소형증발량(mm)"
]

RANDOM_STATE = 42
TEST_DAYS = 30  # 마지막 N일을 테스트로 (데이터 길이에 맞게 조정)



In [26]:
# 1 데이터 로드
df = pd.read_csv(DATA_PATH)
df[COL_DATE] = pd.to_datetime(df[COL_DATE])
df = df.sort_values([COL_GROUP, COL_DATE]).reset_index(drop=True)

# 타입 보정(안전)
df[COL_GROUP] = df[COL_GROUP].astype(str)

In [27]:
# 2 (중요) 날짜 누락 보정: 지역별로 "일별" 캘린더 완성
def make_daily_panel(df_in: pd.DataFrame) -> pd.DataFrame:
    out = []
    for code, g in df_in.groupby(COL_GROUP, sort=False):
        g = g.sort_values(COL_DATE).set_index(COL_DATE)

        full_idx = pd.date_range(g.index.min(), g.index.max(), freq="D")
        g2 = g.reindex(full_idx)
        g2[COL_GROUP] = code
        g2.index.name = COL_DATE
        out.append(g2.reset_index())

    panel = pd.concat(out, ignore_index=True)

    # 날씨는 일별로 존재해야 정상. 누락이 있다면 보간/ffill 정책이 필요할 수 있음.
    # 여기서는 "결측이 생긴 행은 학습에 쓰지 않는다" 원칙으로 유지.
    return panel

panel = make_daily_panel(df)

# 기본 컬럼만 유지(정합성)
keep_cols = [COL_GROUP, COL_DATE] + WEATHER_COLS + [COL_TARGET]
panel = panel[keep_cols]

In [28]:
# 3 공통: 시간 기반 split (누수 방지)
max_date = panel[COL_DATE].max()
cutoff_date = max_date - pd.Timedelta(days=TEST_DAYS)

train_base = panel[panel[COL_DATE] <= cutoff_date].copy()
test_base  = panel[panel[COL_DATE] >  cutoff_date].copy()

print("Train range:", train_base[COL_DATE].min(), "~", train_base[COL_DATE].max())
print("Test  range:", test_base[COL_DATE].min(),  "~", test_base[COL_DATE].max())
print("Train rows:", len(train_base), "Test rows:", len(test_base))

Train range: 2022-06-28 00:00:00 ~ 2023-12-29 00:00:00
Test  range: 2023-12-30 00:00:00 ~ 2024-01-28 00:00:00
Train rows: 219228 Test rows: 13980


In [29]:
# 4) 베이스라인 데이터 구성: "날씨 변수만" + 지역코드 (+ 날짜/지역 메타 보존)
def make_baseline_dataset(df_in: pd.DataFrame):
    cols = [COL_GROUP] + WEATHER_COLS + [COL_TARGET]

    data = df_in[cols].copy()
    meta = df_in[[COL_GROUP, COL_DATE]].copy()  # ⭐ 날짜/지역 보존

    mask = data.notna().all(axis=1)
    data = data[mask].reset_index(drop=True)
    meta = meta[mask].reset_index(drop=True)

    return data, meta

train_bl, train_bl_meta = make_baseline_dataset(train_base)
test_bl,  test_bl_meta  = make_baseline_dataset(test_base)

print("Baseline train/test:", train_bl.shape, test_bl.shape)

Baseline train/test: (167894, 9) (13514, 9)


In [30]:
# 5) Compare (FE) 데이터 구성: Lag/Rolling 생성 (지역별, 일별 기준)
#    + 날짜/지역 메타 보존
def make_fe_dataset(df_in: pd.DataFrame):
    d = df_in.sort_values([COL_GROUP, COL_DATE]).copy()

    # lag
    d["power_lag_1d"] = d.groupby(COL_GROUP)[COL_TARGET].shift(1)
    d["power_lag_7d"] = d.groupby(COL_GROUP)[COL_TARGET].shift(7)

    # rolling (현재일 포함 방지: shift(1) 후 rolling)
    past = d.groupby(COL_GROUP)[COL_TARGET].shift(1)
    d["power_roll_mean_7d"] = (
        past.groupby(d[COL_GROUP])
            .rolling(7)
            .mean()
            .reset_index(level=0, drop=True)
    )
    d["power_roll_std_7d"] = (
        past.groupby(d[COL_GROUP])
            .rolling(7)
            .std()
            .reset_index(level=0, drop=True)
    )

    # 모델 입력용 컬럼
    feature_cols = [COL_GROUP] + WEATHER_COLS + [
        "power_lag_1d",
        "power_lag_7d",
        "power_roll_mean_7d",
        "power_roll_std_7d",
    ]

    data = d[feature_cols + [COL_TARGET]].copy()
    meta = d[[COL_GROUP, COL_DATE]].copy()   # ⭐ 날짜/지역 보존

    # 동일 행 유지용 결측 마스크
    mask = data.notna().all(axis=1)
    data = data[mask].reset_index(drop=True)
    meta = meta[mask].reset_index(drop=True)

    return data, meta

In [31]:

train_fe, train_fe_meta = make_fe_dataset(train_base)
test_fe,  test_fe_meta  = make_fe_dataset(test_base)

print("FE train/test:", train_fe.shape, test_fe.shape)

FE train/test: (120750, 13) (6990, 13)


In [32]:
# 전처리기 + DNN 모델 (베이스라인과 동일 구조로 학습)
#    - 지역코드: OneHot
#    - 수치: StandardScaler
def build_preprocessor(X: pd.DataFrame) -> ColumnTransformer:
    cat_cols = [COL_GROUP]
    num_cols = [c for c in X.columns if c not in cat_cols]

    pre = ColumnTransformer(
        transformers=[
            ("onehot", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("scaler", StandardScaler(), num_cols),
        ]
    )
    return pre

def build_dnn(input_dim: int) -> tf.keras.Model:
    model = Sequential([
        Input((input_dim,)),
        Dense(256, activation="relu"),
        Dropout(0.1),
        Dense(128, activation="relu"),
        Dense(64, activation="relu"),
        Dropout(0.1),
        Dense(32, activation="relu"),
        Dense(1)  # 회귀: 보통 linear(activation 없음) 권장
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


def train_eval(train_df: pd.DataFrame, test_df: pd.DataFrame, run_name: str):
    X_train = train_df.drop(columns=[COL_TARGET])
    y_train = train_df[COL_TARGET].values

    X_test  = test_df.drop(columns=[COL_TARGET])
    y_test  = test_df[COL_TARGET].values

    pre = build_preprocessor(X_train)
    X_train_p = pre.fit_transform(X_train)
    X_test_p  = pre.transform(X_test)

    # sparse -> dense (DNN 입력)
    X_train_p = X_train_p.toarray()
    X_test_p  = X_test_p.toarray()

    model = build_dnn(X_train_p.shape[1])
    es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

    model.fit(
        X_train_p, y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=256,
        callbacks=[es],
        verbose=0
    )

    pred = model.predict(X_test_p, verbose=0).reshape(-1)

    mae  = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)

    return {
        "run": run_name,
        "MAE": float(mae),
        "RMSE": float(rmse),
        "train_rows": int(len(train_df)),
        "test_rows": int(len(test_df)),
        "n_features_raw": int(X_train.shape[1]),
    }


In [33]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

def train_eval_with_pred(train_df: pd.DataFrame, test_df: pd.DataFrame, run_name: str):
    X_train = train_df.drop(columns=[COL_TARGET])
    y_train = train_df[COL_TARGET].values

    X_test  = test_df.drop(columns=[COL_TARGET])
    y_test  = test_df[COL_TARGET].values

    pre = build_preprocessor(X_train)
    X_train_p = pre.fit_transform(X_train).toarray()
    X_test_p  = pre.transform(X_test).toarray()

    model = build_dnn(X_train_p.shape[1])
    es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

    model.fit(
        X_train_p, y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=256,
        callbacks=[es],
        verbose=0
    )

    pred = model.predict(X_test_p, verbose=0).reshape(-1)

    mae  = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)

    result = {
        "run": run_name,
        "MAE": float(mae),
        "RMSE": float(rmse),
        "train_rows": int(len(train_df)),
        "test_rows": int(len(test_df)),
        "n_features_raw": int(X_train.shape[1]),
    }
    return result, y_test, pred


In [34]:
# 실행: Baseline vs FE 비교
import os
import pandas as pd

os.makedirs("outputs", exist_ok=True)

# 1) Baseline vs FE 학습 + 예측
res_bl, y_test_bl, pred_bl = train_eval_with_pred(
    train_bl, test_bl, "baseline_weather_only"
)
res_fe, y_test_fe, pred_fe = train_eval_with_pred(
    train_fe, test_fe, "fe_weather_plus_lag_roll"
)

# 2) 비교표 출력
compare = pd.DataFrame([res_bl, res_fe]).sort_values("RMSE").reset_index(drop=True)
display(compare)

# 3) 예측 결과 저장 (03-2_visualization에서 사용)
out_bl = test_bl_meta.copy()
out_bl["y_true"] = y_test_bl
out_bl["y_pred"] = pred_bl
out_bl.to_csv("outputs/pred_baseline.csv", index=False)

out_fe = test_fe_meta.copy()
out_fe["y_true"] = y_test_fe
out_fe["y_pred"] = pred_fe
out_fe.to_csv("outputs/pred_fe.csv", index=False)

print("Saved files:",
      os.path.exists("outputs/pred_baseline.csv"),
      os.path.exists("outputs/pred_fe.csv"))

,run,MAE,RMSE,train_rows,test_rows,n_features_raw
0,fe_weather_plus_lag_roll,14937.560624,42931.841032,120750,6990,12
1,baseline_weather_only,80262.580726,137869.031114,167894,13514,8


Saved files: True True


In [35]:
compare.to_csv("outputs/compare_metrics.csv", index=False)

In [36]:
# 이 템플릿이 “비교 모델”을 만드는 방식 요약

# Baseline: 전체코드 + 날씨변수들 → 파워

# FE 모델: 전체코드 + 날씨변수들 + (파워 lag/rolling) → 파워

# Split은 **날짜 기준(마지막 N일 테스트)**으로 고정하여 누수 방지 + 공정 비교

# 지역별로 날짜 누락이 있어도 일별 캘린더를 먼저 완성하므로
# lag_1d가 진짜 “전일” 의미를 가집니다.

In [37]:
# 1. 결과 요약 (한 줄 결론)

# Lag / Rolling Feature를 추가한 FE 모델은
# 베이스라인 대비 예측 오차를 약 70~80% 이상 감소시켰다.

# 2. 수치 비교 해석 (정량)
# 구분	MAE	RMSE
# Baseline (날씨만)	71,428	137,039
# FE (날씨 + Lag/Roll)	14,999	41,465
# 개선 폭 (대략)

# MAE 감소율

# (
# 71
# ,
# 428
# −
# 14
# ,
# 999
# )
# /
# 71
# ,
# 428
# ≈
# 79
# %
# 감소
# (71,428−14,999)/71,428≈79%감소

# RMSE 감소율

# (
# 137
# ,
# 039
# −
# 41
# ,
# 465
# )
# /
# 137
# ,
# 039
# ≈
# 70
# %
# 감소
# (137,039−41,465)/137,039≈70%감소

# ➡️ 이 정도 차이는 “미세 개선”이 아니라 “문제 정의를 제대로 잡았다”는 신호입니다.

# 3. 왜 이렇게 큰 차이가 났는가? (핵심 해석)
# ① 베이스라인의 한계 (날씨만 사용)

# 전력 사용량은:

# 날씨 영향도 받지만

# 어제·지난주 사용량의 자기상관성이 훨씬 큼

# 날짜/과거 정보 없이:

# “오늘 기온이 -5도면 전력은 이쯤”
# → 불안정한 평균 추정만 가능

# ➡️ 그래서 오차가 큼

# ② FE 모델이 잘한 것 (Lag / Rolling)

# 추가된 핵심 정보는 사실 이것입니다:

# power_lag_1d : 어제 전력

# power_lag_7d : 지난주 같은 요일 전력

# power_roll_mean_7d : 최근 사용 패턴 평균

# power_roll_std_7d : 변동성

# ➡️ 모델 입장에서는:

# “오늘 전력 = 어제 전력 ± 날씨 영향 ± 최근 추세”

# 이게 전력 수요 예측의 정석 공식입니다.

# 4. train_rows / test_rows 차이에 대한 해석 (중요)
# 구분	train_rows	test_rows
# Baseline	167,894	13,514
# FE	120,750	6,990

# 이 차이는 문제가 아니라 정상입니다.

# 이유

# Lag / Rolling 생성 시:

# 각 지역별로 초기 7일 + 누락일 제거

# 날짜 패널 보정 과정에서 결측 제거

# ➡️ “데이터를 희생해서 정보 밀도를 높인” 전형적인 시계열 처리

# 오히려:

# 적은 데이터로 훨씬 좋은 성능
# → 모델 신뢰도 상승

# 5. 이 결과를 보고서에 쓰는 “정석 문장”

# 아래 문장은 그대로 써도 됩니다.

# 날씨 변수만을 사용한 베이스라인 모델은 전력 사용량의 시계열적 특성을 반영하지 못해 높은 예측 오차를 보였다.
# 반면, 지역별 일별 전력 사용량에 대해 Lag 및 Rolling Feature를 추가한 모델은 과거 사용 패턴을 효과적으로 학습함으로써 MAE와 RMSE를 각각 약 79%, 70% 이상 개선하였다.
# 이는 전력 수요 예측 문제에서 과거 사용량 기반 특징이 핵심적인 설명력을 가진다는 점을 정량적으로 입증한 결과이다.

# 6. 지금 단계에서 “이미 확보한 것”

# 이 프로젝트 기준으로, 당신은 이미 다음을 확보했습니다:

# ✅ 공정한 Baseline vs FE 비교 실험

# ✅ 시계열 자기상관의 중요성에 대한 정량 증거

# ✅ 데이터 누수 방지 구조(날짜 기준 split)

# ✅ 실무에서 바로 쓰는 Feature 구성

# 👉 이 상태만으로도 “EDA + Feature Engineering + ML 모델링” 파트는 완성입니다.

# 7. 다음 단계 추천 (우선순위)